In [1]:
using ADIOS2
using Plots

Thu Sep  8 11:54:49 2022: [unset]:_pmi_alps_init:alps_get_placement_info returned with error -1
Thu Sep  8 11:54:49 2022: [unset]:_pmi_init:_pmi_alps_init returned -1
┌ Warning: Error requiring `MPICH_jll` from `MPI`
│   exception = (ErrorException("MPICH_jll cannot be loaded: MPI.jl is configured to use the system MPI library"), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00001555460bdf6e, Ptr{Nothing} @0x0000155553db1459, Ptr{Nothing} @0x0000155553dcdc05, Ptr{Nothing} @0x0000155553dcd8be, Ptr{Nothing} @0x0000155553dce501, Ptr{Nothing} @0x0000155553dcef87, Base.InterpreterIP in top-level CodeInfo for MPI at statement 8, Ptr{Nothing} @0x0000155553dea841, Ptr{Nothing} @0x0000155553dec659, Ptr{Nothing} @0x000015553f7a8831, Ptr{Nothing} @0x000015553f7a885c, Ptr{Nothing} @0x0000155553db1459, Ptr{Nothing} @0x0000155553dcdc05, Ptr{Nothing} @0x0000155553dcd8be, Ptr{Nothing} @0x0000155553dce595, Ptr{Nothing} @0x0000155553dce880, Ptr{Nothing} @0x0000155553dced3a, Base.InterpreterIP 

In [2]:
adios = adios_init_serial("adios2.xml")                                                    # Use the configurations defined in "adios2.xml"...
io = declare_io(adios, "IO")                                                               # ... in the section "writerIO"
engine = open(io, "diffusion2D.bp", mode_read)                                             # Open the file/stream "diffusion2D.bp"
engine_type = type(engine)                                                                 # Check the type of the engine: BP4/SST..

"BP4Reader"

In [3]:
n = 0                                                                                      # counter for while loop
nx, ny     = 128, 128                                                                      # Number of gridpoints in dimensions x and y
T = zeros(nx-2, ny-2)                                                              # Prealocate memory for T using the hard code data
# setting up Animation
gr()
ENV["GKSwstype"]="nul"
anim = Animation();

In [4]:
inquire_all_variables(io::AIO)                                                             # There are no available variables before the first step when using SST/BP4

2-element Vector{Variable}:
 Variable(heat capacity)
 Variable(temperature)

In [ ]:
while begin_step(engine, step_mode_read, 100.0) != step_status_end_of_stream::StepStatus   # Begin the next ADIOS2 read step while there is any
    if n % 2 == 1
        T_id = inquire_variable(io, "temperature")                                         # Get a handle for the variable T. ATTENTION: T_id is only valid for the current step when using streaming (SST or file streaming); thus, moving it into the if statement below would lead to an error!
        get(engine, T_id, T)                                                               # Add T to variable(s) for reading (SetSelection() would be needed in addition if we were reading the data with multiple MPI processes: this would make sure that every process reads \"his\" block)
        perform_gets(engine)                                                               # Perform Get with the variables in 
        heatmap(transpose(T), aspect_ratio=1); frame(anim);                                # Plot the temperature using animation
    end
    end_step(engine)                                                                       # End Step
    n += 1
end
println("done")

In [5]:
mp4(anim, "diffusion2D_outviz.mp4", fps = 15)

┌ Info: Saved animation to 
│   fn = /users/jsebin/adios2-tutorial/example/final/diffusion2D/diffusion2D_outviz.mp4
└ @ Plots /apps/daint/UES/jenkins/7.0.UP03/21.09/daint-gpu/software/JuliaExtensions/1.6.3-CrayGNU-21.09-cuda/extensions/packages/Plots/5kcBO/src/animation.jl:114


Plots.AnimatedGif("/users/jsebin/adios2-tutorial/example/final/diffusion2D/diffusion2D_outviz.mp4")